In [1]:
import sqlite3
import pandas as pd

In [2]:
! pip install pandasql

In [3]:
conn = sqlite3.connect('students.db')
conn.execute('pragma foreign_keys=ON')
cur = conn.cursor()

In [4]:
def display_as_table(data, headers):
    df = pd.DataFrame(data=data, columns=[i[0] for i in headers])
    return df

In [5]:
# A Primary Key satifies 3 conditions:
#1) Any column (or collection of colums) that are unique
#2) not Null
#3)Each Table can have, at most,1 primary key (column)



In [6]:
#cur.execute("DROP TABLE SI")

In [7]:
statement = '''
CREATE TABLE SI(
Name varcchar(255) NOT NULL,
GPA float CHECK (GPA <= 4 and GPA >=0),
Major varchar(255) NOT NULL,
Year integer DEFAULT 1,
CONSTRAINT name_pk PRIMARY KEY (Name)
);'''

cur.execute(statement)

OperationalError: table SI already exists

In [8]:
cur.execute("SELECT * FROM SI")
display_as_table(cur.fetchall(),cur.description)

,Name,GPA,Major,Year


In [9]:
cur.execute("INSERT INTO SI VALUES ('Yoda', 3.5, 'Physics', 3)")
cur.execute("INSERT INTO SI VALUES ('Vader', 3.2, 'Math', 3)")
cur.execute("INSERT INTO SI VALUES ('Kyla', 3.9, 'Physics', 2)")

In [10]:
cur.execute("SELECT * FROM SI")
display_as_table(cur.fetchall(),cur.description)

,Name,GPA,Major,Year
0,Yoda,3.5,Physics,3
1,Vader,3.2,Math,3
2,Kyla,3.9,Physics,2


In [11]:
cur.execute("INSERT INTO SI VALUES('Yoda', 3.2, 'CS', 2)")

IntegrityError: UNIQUE constraint failed: SI.Name

In [12]:
cur.execute("INSERT INTO SI (GPA, Major, Year) VALUES(3.2, 'CS', 2)")

IntegrityError: NOT NULL constraint failed: SI.Name

In [ ]:
# Examples the one can't put info into the DB without a primary key

In [ ]:
# You can also have MULTIPLE Key Columns ie 
# I could have used:  CONSTRAINT name_major_year_pk PRIMARY KEY (Name,Major, Year)

#In that instance no one person can have identical name, major, AND year

In [ ]:
#Auto-increpementing Integer Primary Key 

In [13]:
statement = '''
CREATE TABLE SI_2 (
Student_ID integer NOT NULL PRIMARY KEY AUTOINCREMENT,
Name varcchar(255) NOT NULL,
GPA float CHECK (GPA <= 4 and GPA >=0),
Major varchar(255) NOT NULL,
Year integer DEFAULT 1
);'''

cur.execute(statement)

In [14]:
cur.execute("INSERT INTO SI_2 (Name, GPA, Major, Year) VALUES ('Yoda', 3.5, 'Physics', 3)")
cur.execute("INSERT INTO SI_2 (Name, GPA, Major, Year) VALUES ('Vader', 3.2, 'Math', 3)")
cur.execute("INSERT INTO SI_2 (Name, GPA, Major, Year) VALUES ('Kyla', 3.9, 'Physics', 2)")

In [15]:
cur.execute("SELECT * FROM SI_2")
display_as_table(cur.fetchall(),cur.description)

,Student_ID,Name,GPA,Major,Year
0,1,Yoda,3.5,Physics,3
1,2,Vader,3.2,Math,3
2,3,Kyla,3.9,Physics,2


In [ ]:
# Foreign Key

In [16]:
cur.execute('DROP TABLE Students')

In [17]:
cur.execute ("DROP TABLE Enrollment")

OperationalError: no such table: Enrollment

In [18]:
statement = '''
CREATE TABLE Students(
Name varcchar(255) NOT NULL,
GPA float CHECK (GPA <= 4 and GPA >=0),
Major varchar(255) NOT NULL,
Year integer DEFAULT 1,
CONSTRAINT name_pk PRIMARY KEY (Name)
);'''

cur.execute(statement)

In [19]:
cur.execute("INSERT INTO Students VALUES ('Yoda', 3.5, 'Physics', 3)")
cur.execute("INSERT INTO Students VALUES ('Vader', 3.2, 'Math', 3)")
cur.execute("INSERT INTO Students VALUES ('Kylo', 3.9, 'Physics', 2)")

In [20]:
cur.execute("SELECT * FROM Students")
display_as_table(cur.fetchall(), cur.description)

,Name,GPA,Major,Year
0,Yoda,3.5,Physics,3
1,Vader,3.2,Math,3
2,Kylo,3.9,Physics,2


In [21]:
# create Enrollment table
statement = '''
CREATE TABLE Enrollment (
CourseName varchar(255) NOT NULL,
StudentName varchar(255) NOT NULL,
FOREIGN KEY (StudentName) REFERENCES Students(Name)
);'''
cur.execute(statement)

In [22]:
cur.execute("INSERT INTO ENROLLMENT VALUES ('Thermo', 'Yoda')")
cur.execute("INSERT INTO ENROLLMENT VALUES ('Thermo', 'Kylo')")
cur.execute("INSERT INTO ENROLLMENT VALUES ('Calc', 'Vader')")


In [23]:
cur.execute("SELECT * FROM Enrollment")
display_as_table(cur.fetchall(), cur.description)

,CourseName,StudentName
0,Thermo,Yoda
1,Thermo,Kylo
2,Calc,Vader


In [ ]:
#Should not be allowed : Adding Student to Enrollment table who does not exist

In [24]:
cur.execute("INSERT INTO Enrollment VALUES ('Calc', 'Jim')")

IntegrityError: FOREIGN KEY constraint failed

In [ ]:
cur.execute("SELECT * FROM Enrollment")
display_as_table(cur.fetchall(), cur.description)

In [ ]:
#Should not be allowed : Delete a student from Students if it leaves

In [25]:
cur.execute("DELETE FROM Students WHERE Name = 'Kylo'")

IntegrityError: FOREIGN KEY constraint failed

In [26]:
cur.execute("SELECT * FROM Enrollment")
display_as_table(cur.fetchall(), cur.description)

,CourseName,StudentName
0,Thermo,Yoda
1,Thermo,Kylo
2,Calc,Vader


In [27]:
cur.execute("SELECT * FROM Students")
display_as_table(cur.fetchall(), cur.description)

,Name,GPA,Major,Year
0,Yoda,3.5,Physics,3
1,Vader,3.2,Math,3
2,Kylo,3.9,Physics,2


In [28]:
#How to put them in?

In [29]:
cur.execute("INSERT INTO Students VALUES ('Rey', 3.2, 'Math', 2 )")
cur.execute("INSERT INTO Enrollment VALUES('Calc', 'Rey')")

In [30]:
cur.execute("SELECT * FROM Students")
display_as_table(cur.fetchall(), cur.description)

,Name,GPA,Major,Year
0,Yoda,3.5,Physics,3
1,Vader,3.2,Math,3
2,Kylo,3.9,Physics,2
3,Rey,3.2,Math,2


In [31]:
cur.execute("SELECT * FROM Enrollment")
display_as_table(cur.fetchall(), cur.description)

,CourseName,StudentName
0,Thermo,Yoda
1,Thermo,Kylo
2,Calc,Vader
3,Calc,Rey
